### 03_load_saved_model.ipynb is a smoke test 

that verifies the exported model loads correctly, accepts inputs in the expected format, and produces valid outputs.

Nothing more, nothing less.

SavedModel (.pb) loads without error

Input shape and preprocessing are correct

Inference runs in production-style (inference-only) mode

Output structure and dimensions match expectations

In [1]:
import tensorflow as tf
import numpy as np
import pathlib
import json

print("TensorFlow:", tf.__version__)

2026-01-16 21:47:20.428131: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-16 21:47:23.917914: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow: 2.20.0-dev0+selfbuilt


In [2]:
tf.keras.utils.set_random_seed(42)

In [3]:
DATA_DIR = pathlib.Path("/mnt/d/data/food101")
TEST_DIR = DATA_DIR / "test"

SAVED_MODEL_DIR = pathlib.Path(
    "/mnt/d/04_Food101-EfficientNetV2S-model/models/export/food101_savedmodel"
)

# should be the same as used during training
IMG_SIZE = (384, 384)
BATCH_SIZE = 32



Keras 3 treats TensorFlow SavedModel as an inference-only graph, not a full Keras model, so it cannot be loaded directly with load_model().
TFSMLayer explicitly wraps the SavedModel and specifies which exported function (serving_default) to call.
Wrapping it in Sequential provides a Keras-compatible model interface for evaluation and prediction.

In [4]:
from keras.layers import TFSMLayer

model = tf.keras.Sequential([
    TFSMLayer(SAVED_MODEL_DIR.as_posix(), 
              call_endpoint="serving_default")
])

model.summary()

I0000 00:00:1768596446.104664   24512 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5518 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:64:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tfsm_layer (TFSMLayer)          │ ?                      │    20,590,021 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,590,021 (78.54 MB)

 Trainable params: 20,436,149 (77.96 MB)

 Non-trainable params: 153,872 (601.06 KB)

Since the model was trained using TFDS, the evaluation dataset is reloaded from TFDS (food101 test split) to ensure consistent preprocessing, label ordering, and deterministic evaluation.

In [5]:
import tensorflow_datasets as tfds
(ds_train, ds_val), ds_info = tfds.load(
                                        "food101",
                                        split=["train", "validation"],
                                        shuffle_files=False,
                                        as_supervised=True,
                                        with_info=True,
                                    )
ds_val.element_spec

(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [6]:
ds_info

tfds.core.DatasetInfo(
    name='food101',
    full_name='food101/2.0.0',
    description="""
    This dataset consists of 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. On purpose, the training images were not cleaned, and thus still contain some amount of noise. This comes mostly in the form of intense colors and sometimes wrong labels. All images were rescaled to have a maximum side length of 512 pixels.
    """,
    homepage='https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/',
    data_dir='/home/raghu/tensorflow_datasets/food101/2.0.0',
    file_format=tfrecord,
    download_size=4.65 GiB,
    dataset_size=4.77 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
   

In [7]:
class_names = ds_info.features["label"].names
num_classes = ds_info.features["label"].num_classes

In [8]:
IMG_SIZE = (384, 384)
BATCH_SIZE = 32

def preprocess_fn(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)
    image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
    return image, label

🚫 No augmentation
🚫 No shuffle
🚫 No randomness

In [9]:
test_ds = (
    ds_val
    .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_ds.element_spec

(TensorSpec(shape=(None, 384, 384, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

a TensorFlow SavedModel exposes named output signatures, not a single return tensor.
When you load it via TFSMLayer, Keras calls the SavedModel’s serving_default function, which returns a dictionary of outputs (e.g., {"outputs": tensor}) instead of a raw tensor—this preserves explicit output names for production and avoids ambiguity when models have multiple outputs.

In [10]:
# Sanity check
images, labels = next(iter(test_ds))
preds = model(images)
print(preds.keys())

print(type(images))
preds_tensor = preds["output_layer"]
print(images.shape, preds_tensor.shape)

2026-01-16 21:47:38.432053: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:396] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2026-01-16 21:47:40.080762: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300


dict_keys(['output_layer'])
<class 'tensorflow.python.framework.ops.EagerTensor'>
(32, 384, 384, 3) (32, 101)
